In [27]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from pydantic.v1 import BaseModel, Field

# Create prompt
template = """
Extract the relevant entities with their properties
And for the property that is not present and not required by the function parameter, Do not include in the output
"""

prompt = ChatPromptTemplate.from_messages(
    {("system", template), ("user", "{input}")}
)

# Create model
model = ChatOpenAI()

# Create schema


class Person(BaseModel):
    name: str
    age: int
    location: str
    hobby: str


# Create tools
pydantic_schemas = [Person]
tools = [convert_to_openai_tool(p) for p in pydantic_schemas]

# Bind the tools to the model
model = model.bind_tools(tools=tools)

# Create the parser
parser = PydanticToolsParser(tools=pydantic_schemas)

# Create LCEL Chain
chain = prompt | model | parser

# Invoke
response = chain.invoke({
    "input": "Happy Lim is 22 year old, and He lived in Malaysia and he like to play basketball"
})

print(response)

[Person(name='Happy Lim', age=22, location='Malaysia', hobby='play basketball')]
